In [2]:
 from __future__ import print_function
import tensorflow as tf
import os
import import_ipynb
from skimage.transform import resize
from skimage.io import imsave
from data import load_test_data , load_train_data

import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, concatenate, Conv2D, MaxPooling2D, Conv2DTranspose,Dense , Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import backend as K
from tensorflow.keras.activations import softmax
 
 
 
 
K.set_image_data_format('channels_last')  # TF dimension ordering in this code
 
img_rows = 96
img_cols = 96
 
smooth = 1.



def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    
 
 
def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
 
def get_unet():
    inputs = Input((img_rows, img_cols, 1))
    print("++++++++++++++++++++++++++++++++++++++++++++++new Line")
    
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    print("dim",conv1.shape)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    print("dim",pool1.shape)
 
        
        
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    print('dim',conv2.shape)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    print('dim',pool2.shape)
   


    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    print('dim',conv3.shape)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    print('dim',pool3.shape)
  
 
    conv4 = Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    print('dim',conv4.shape)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
   
   
    conv5 = Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    print('dim',conv5.shape)
    

 
    up6 = concatenate([Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv5), conv4], axis=3)
    print('dim',up6.shape)
    conv6 = Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    print('dim',conv6.shape)
   
 
    up7 = concatenate([Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv6), conv3], axis=3)
    print('dim',up7.shape)
    conv7 = Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    print('dim',conv7.shape)
   
 
    up8 = concatenate([Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv7), conv2], axis=3)
    print('dim',up8.shape)
    conv8 = Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    print('dim',conv8.shape)
   
 
    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(conv8), conv1], axis=3)
    print('dim',up9.shape)
    conv9 = Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    print('dim',conv9.shape)
    
 
    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)
    print("dim", conv10.shape) 
 
    model = Model(inputs=[inputs], outputs=[conv10])
 
    model.compile(optimizer=Adam(learning_rate=1e-5), loss=[dice_coef_loss],
                  metrics=[dice_coef])
 
    return model
 
def preprocess(imgs):
    imgs_p = np.ndarray((imgs.shape[0], img_rows, img_cols), dtype=np.uint8)
    for i in range(imgs.shape[0]):
        imgs_p[i] = resize(imgs[i], (img_cols, img_rows), preserve_range=True)
 
    imgs_p = imgs_p[..., np.newaxis]
    return imgs_p
 
def train_and_predict():
    imgs_train,imgs_mask_train,imgs_value_train=load_train_data()
 
    imgs_train = preprocess(imgs_train)
    imgs_mask_train = preprocess(imgs_mask_train)
 
    imgs_train = imgs_train.astype('float32')
    mean = np.mean(imgs_train)  # mean for data centering
    std = np.std(imgs_train)  # std for data normalization
 
    imgs_train -= mean
    imgs_train /= std
 
    imgs_mask_train = imgs_mask_train.astype('float32')
    imgs_mask_train /= 255.  # scale masks to [0, 1]
 
    print('-'*30)
    print('Creating and compiling model...')
    print('-'*30)
    model = get_unet()
    model_checkpoint = ModelCheckpoint('weights.h5', monitor='val_loss' , mode = 'min' , verbose = 1 , save_best_only=True )
 
    print('-'*30)
    print('Fitting model...')
    print('-'*30)
    model.fit(imgs_train, y=[imgs_mask_train], batch_size=32, epochs= 20, verbose=1, shuffle=True,
              validation_split=0.2 ,  callbacks=[model_checkpoint])
    
    print('-'*30)
    print('Loading and preprocessing test data...')
    print('-'*30)
    imgs_test, imgs_id_test = load_test_data()
    imgs_test = preprocess(imgs_test)

    imgs_test = imgs_test.astype('float32')
    imgs_test -= mean
    imgs_test /= std

    print('-'*30)
    print('Loading saved weights...')
    print('-'*30)
    model.load_weights('weights.h5')

    print('-'*30)
    print('Predicting masks on test data...')
    print('-'*30)
    n_imgs_test = imgs_test.shape[0]
    #for i in range(n_imgs_test):

    #worked for 7 vs 1
    #imgs_mask_test, imgs_value_test= model.predict(np.expand_dims(imgs_test[0] , axis = 0), verbose=1)


    imgs_mask_test = model.predict(imgs_test, verbose=1)
    print("mask final img 1" , imgs_mask_test)
    np.save('imgs_mask_test.npy', imgs_mask_test)

    print('-' * 30)
    print('Saving predicted masks to files...')
    print('-' * 30)
    pred_dir = 'preds'
    
    # print(imgs_mask_test.shape , imgs_value_test.shape)
    if not os.path.exists(pred_dir):
        os.mkdir(pred_dir)
    for image, image_id in zip(imgs_mask_test, imgs_id_test):
        image = (image[:, :, 0] * 255.).astype(np.uint8)
        imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)


 
     
if __name__ == '__main__':
    train_and_predict()
        

------------------------------
Creating and compiling model...
------------------------------
++++++++++++++++++++++++++++++++++++++++++++++new Line
dim (None, 96, 96, 32)
dim (None, 48, 48, 32)
dim (None, 48, 48, 64)
dim (None, 24, 24, 64)
dim (None, 24, 24, 128)
dim (None, 12, 12, 128)
dim (None, 12, 12, 256)
dim (None, 6, 6, 512)
dim (None, 12, 12, 512)
dim (None, 12, 12, 256)
dim (None, 24, 24, 256)
dim (None, 24, 24, 128)
dim (None, 48, 48, 128)
dim (None, 48, 48, 64)
dim (None, 96, 96, 64)
dim (None, 96, 96, 32)
dim (None, 96, 96, 1)
------------------------------
Fitting model...
------------------------------
Epoch 1/20
141/141 [==============================] - 32s 121ms/step - loss: -0.0252 - dice_coef: 0.0252 - val_loss: -0.0198 - val_dice_coef: 0.0199

Epoch 00001: val_loss improved from inf to -0.01976, saving model to weights.h5
Epoch 2/20
141/141 [==============================] - 13s 92ms/step - loss: -0.0288 - dice_coef: 0.0288 - val_loss: -0.0264 - val_dice_coef: 0.02

------------------------------
Saving predicted masks to files...
------------------------------


<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\100_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1000_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1004_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1008_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1012_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1013_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb

<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1180_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1181_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1183_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1185_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1188_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1189_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edb

<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1371_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1372_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1374_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1379_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1380_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\1383_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edb

...

<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5027_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5028_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5029_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5030_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5038_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5039_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edb

<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5214_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5215_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5216_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5217_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5222_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5224_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edb

<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5397_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5398_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\540_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5404_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\5409_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\541_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>

<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\624_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\625_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\627_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\630_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\633_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\635_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183

<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\82_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\824_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\829_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\836_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\839_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: UserWarning: preds\840_pred.png is a low contrast image
  imsave(os.path.join(pred_dir, str(image_id) + '_pred.png'), image)
<ipython-input-2-a03f0833edbb>:183: